In [ ]:
# !yes | pip3 install -r torch==1.10.0
# !yes | pip3 install opencv-python==4.5.3.56
# !yes | pip3 install tqdm==4.62.3
# !yes | pip3 install torchvision==0.11.1
#!yes | pip3 install seaborn==0.11.2

In [75]:
import torch
from PIL import Image
import glob
import os
import pandas as pd
from training_xml_generator import Traning_xml_generator

In [18]:

pre_trained_model_path = "./yolov5_modelo_carimbo_assinatura.pt"
imgsz = 1000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.hub.load('ultralytics/yolov5', 'custom', path=pre_trained_model_path,force_reload=True).to(device)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /home/patrick/.cache/torch/hub/master.zip


requirements: matplotlib>=3.2.2 not found and is required by YOLOv5, attempting auto-update...

requirements: numpy>=1.18.5 not found and is required by YOLOv5, attempting auto-update...

requirements: Pillow>=7.1.2 not found and is required by YOLOv5, attempting auto-update...

requirements: requests>=2.23.0 not found and is required by YOLOv5, attempting auto-update...


YOLOv5 🚀 2021-11-3 torch 1.10.0+cu102 CUDA:0 (NVIDIA GeForce GTX 1650, 3912MiB)

Fusing layers... 
Model Summary: 232 layers, 7249215 parameters, 0 gradients
Adding AutoShape... 



requirements: 4 packages updated per /home/patrick/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



In [ ]:
convert_dict = {
                'xmin':int,'xmax':int,
                'ymin':int,'ymax':int,
                'width':int,'height':int
            }

In [76]:
imgs_path = glob.glob("guias_sistema/guias/*.jpg")
df_all = pd.DataFrame(columns = ['filename','width','height','xmin','ymin','xmax','ymax','score','class_number','class'])
for path in imgs_path:
    img = Image.open(path)
    results = model(img, size=imgsz)  # includes NMS

    img_boxes = results.pandas().xyxy[0]
    img_boxes.columns = ['xmin','ymin','xmax','ymax','score','class_number','class']
    img_boxes['filename'] = os.path.basename(path)
    img_boxes['width'] = img_boxes['xmax'] - img_boxes['xmin']
    img_boxes['height'] = img_boxes['ymax'] - img_boxes['ymin']

    w,h = img.size
    Traning_xml_generator(path,img_boxes.astype(convert_dict),w,h,1,'guias_sistema/out/').generator()
    df_all = pd.concat([df_all, img_boxes])
    

In [72]:

df_all = df_all.astype(convert_dict)

In [73]:
df_all.drop(columns=['score','class_number'],inplace=True)
df_all.head(10) 

,filename,width,height,xmin,ymin,xmax,ymax,class
0,1_0678.jpg,451,588,2951,5079,3402,5668,carimbo_med
1,1_0678.jpg,776,565,2745,5185,3522,5750,assinatura_med


In [74]:
df_all.to_csv('label_map.csv', index=False)

In [ ]:

gerador  = Traning_xml_generator(path,boxes_array,w,h,1,'guias_sistema/out/')

## Exportando para yolov5

In [53]:
imgs_path = glob.glob("guias_sistema/guias/*.jpg")
for path in imgs_path:
    img = Image.open(path)
    results = model(img, size=imgsz)  # includes NMS

    img_boxes = results.pandas().xyxy[0]
    img_boxes.columns = ['x1','y1','x2','y2','score','class_number','class']
    # print(img_boxes)


    h,w = img.size
    info_dict = {'bboxes': [],
                'filename': os.path.basename(path), 
                'image_size': (w, h , 3)}
    
    for n,r in img_boxes.iterrows():
    
        info_dict['bboxes'].append({'class_n': r.class_number,
                                'xmin': r.x1,
                                'ymin': r.y1,
                                'xmax': r.x2, 
                                'ymax': r.y2
                                })

    # print(info_dict)
    convert_to_yolov5(info_dict)

In [52]:
def convert_to_yolov5(info_dict):
    print_buffer = []
    
    # For each bounding box
    for b in info_dict["bboxes"]:
        
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(b["class_n"], b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join("guias_sistema/guias", info_dict["filename"].replace("jpg", "txt"))
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))